In [13]:
from pyspark.sql import SparkSession, functions as F
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import pandas as pd

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("Process Cleaned JSON Data").getOrCreate()

df = spark.read.json("/home/yevhen/Проекти/butsa/json/filtered_players.json")

print(df.count())

filtered_players = (
    df.filter(
        (df.primaryPosition == "Cf")  # |
        # (df.primaryPosition == "Cm") |
        # (df.primaryPosition == "Lm") |
        # (df.primaryPosition == "Rm")  |
        # (df.primaryPosition == "Am")
    )
    .sort(df.rTalent.desc(), df.power.desc())
    .select(
        "id",
        "name",
        "country_name",
        "primaryPosition",
        "secondaryPosition",
        "club",
        "rTalent",
        "power",
        "team_id",
    )
)

# Виводимо результати
filtered_players.show(300, truncate=False)

# Перетворюємо Spark DataFrame у Pandas DataFrame
pandas_df = filtered_players.toPandas()

# Зберігаємо у файл Excel
output_path = "/home/yevhen/Проекти/butsa/json/filtered_players.xlsx"
# with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
#     pandas_df.to_excel(writer, sheet_name="Forwards", index=False)

wb = load_workbook(output_path)
ws = wb["Forwards"]  # Відкриваємо потрібний лист

# Форматування клітинок: заповнення кольором, якщо rTalent > 4.5
fill_highlight = PatternFill(
    start_color="ADD8E6", end_color="ADD8E6", fill_type="solid"
)  # ADD8E6 - light blue, FF7F7F - light red

for row in ws.iter_rows(min_row=2, max_row=ws.max_row):  # Починаємо з другого рядка
    club_cell = row[
        6 - 1
    ]  # Індекс колонки (мінус 1, бо індекси Python починаються з 0)
    if club_cell.value == "Виллем II":  # Ницца, Виллем II
        for cell in row:
            cell.fill = fill_highlight

# Зберігаємо файл
wb.save(output_path)

283
+------+---------------------+-------------------+---------------+-----------------+---------------------+-------+-----+---------------------------------+
|id    |name                 |country_name       |primaryPosition|secondaryPosition|club                 |rTalent|power|team_id                          |
+------+---------------------+-------------------+---------------+-----------------+---------------------+-------+-----+---------------------------------+
|851766|Альваро Мората       |Испания            |Cf             |                 |Вильярреал           |5.1    |24.98|https://www.butsa.ru/roster/305  |
|853108|Фидель Боливар       |Венесуэла          |Cf             |Am               |Кильмес              |5.0    |40.24|https://www.butsa.ru/roster/10743|
|851955|Басилио Рубиньо      |Боливия            |Cf             |Am               |Тулуза               |5.0    |35.18|https://www.butsa.ru/roster/698  |
|852992|Зорепад Калапач      |Украина            |Cf             |

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("Average rTalent by Club").getOrCreate()

# Завантаження JSON у Spark DataFrame
df = spark.read.json("/home/yevhen/Проекти/butsa/json/filtered_players.json")

# Обчислення середнього rTalent по Club
avg_rTalent_by_club = (
    df.groupBy("club")  # Групуємо за країною та клубом
    .agg(
        F.avg("rTalent").alias("avg_rTalent"),  # Обчислюємо середнє значення rTalent
        F.avg("power").alias("avg_power"),  # Обчислюємо середнє значення power
        F.count("id").alias("count"),  # Рахуємо кількість гравців у клубі
    )
    .filter(F.col("count") >= 6)
    .select("club", "avg_rTalent", "avg_power", "count")
    .orderBy(F.desc("avg_rTalent"))  # Сортуємо за спаданням середнього rTalent
)

# Виводимо результати
avg_rTalent_by_club.show(40, truncate=False)

+--------------+------------------+------------------+-----+
|club          |avg_rTalent       |avg_power         |count|
+--------------+------------------+------------------+-----+
|Кильмес       |4.916666666666667 |33.115            |6    |
|Солфорд       |4.699999999999999 |21.651666666666667|6    |
|Аль Барк Тарим|4.511111111111111 |31.845555555555556|9    |
|Спартак Москва|4.51              |33.352            |10   |
|Уракан Манати |4.4818181818181815|28.025454545454544|11   |
|Газ Метан     |4.477777777777778 |37.26444444444444 |9    |
|Виллем II     |4.441666666666667 |25.08             |12   |
|АнтА          |4.325             |23.947499999999998|8    |
|ПФК Рось      |4.300000000000001 |27.415000000000003|10   |
|Ницца         |4.238461538461538 |21.11538461538461 |13   |
|Х.Б.С.        |4.166666666666667 |29.636666666666667|6    |
+--------------+------------------+------------------+-----+



In [14]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive


def upload_excel_to_google_drive(
    file_path,
    file_name,
    mime_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
):
    """
    Завантажує Excel-файл у Google Диск як Google Sheet.
    Якщо файл із такою назвою вже існує, він буде перезаписаний.

    :param file_path: Шлях до локального файлу Excel.
    :param file_name: Назва файлу, який буде створено або перезаписано на Google Диску.
    :param mime_type: Тип файлу для завантаження (за замовчуванням Google Sheet).
    """
    # Авторизація через GoogleAuth
    gauth = GoogleAuth()
    gauth.LoadClientConfigFile(
        "/home/yevhen/Проекти/butsa/json/client_secret_935743862394-45jpsgc3uv5g890ivtfj5843hj6p4u65.apps.googleusercontent.com.json"
    )
    gauth.LocalWebserverAuth()  # Відкриває браузер для авторизації
    drive = GoogleDrive(gauth)

    # Пошук файлу з такою ж назвою
    file_list = drive.ListFile(
        {"q": f"title='{file_name}' and trashed=false"}
    ).GetList()

    if file_list:
        # Якщо файл знайдено, оновлюємо його
        file = file_list[0]  # Беремо перший файл із такою назвою
        file.SetContentFile(file_path)
        file.Upload()
        print(f"Файл '{file_name}' успішно оновлено на Google Диску.")
    else:
        # Якщо файл не знайдено, створюємо новий
        file = drive.CreateFile({"title": file_name, "mimeType": mime_type})
        file.SetContentFile(file_path)
        file.Upload()


upload_excel_to_google_drive(
    file_path="/home/yevhen/Проекти/butsa/json/filtered_players.xlsx",
    file_name="Filtered players",
)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=935743862394-45jpsgc3uv5g890ivtfj5843hj6p4u65.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
Файл 'Filtered players' успішно оновлено на Google Диску.


In [3]:
import json
from collections import defaultdict
import os

def calculate_division_strength(file_paths):
    # ключ — divisionid (str), значення — сума power і кількість гравців
    division_stats = defaultdict(lambda: {"total_power": 0.0, "player_count": 0})

    for file_path in file_paths:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Проходимо всі команди у файлі
        for team_data in data.get("teams", {}).values():
            division_id = team_data.get("divisionid")
            if not division_id:  # якщо нема divisionid — пропускаємо
                continue

            for player_data in team_data.get("players", {}).values():
                power = player_data.get("power")
                try:
                    power = float(power)
                except (TypeError, ValueError):
                    continue

                division_stats[division_id]["total_power"] += power
                division_stats[division_id]["player_count"] += 1

    # Рахуємо середню силу по кожному divisionid
    division_avg_power = {
        div_id: stats["total_power"] / stats["player_count"]
        for div_id, stats in division_stats.items()
        if stats["player_count"] > 0
    }
    return division_avg_power, division_stats  # повернемо ще й лічильники

# Список файлів з країнами
country_files_dir = "/home/yevhen/Проекти/butsa/json/cleaned_responses/"
country_files = [
    os.path.join(country_files_dir, f)
    for f in os.listdir(country_files_dir)
    if f.endswith(".txt")
]

division_strengths, division_stats = calculate_division_strength(country_files)

# Виведення: відсортовано за середньою силою спадно
for div_id, avg_power in sorted(division_strengths.items(), key=lambda x: x[1], reverse=True):
    n = division_stats[div_id]["player_count"]
    print(f"Division {div_id}: Average Power = {avg_power:.2f} (players={n})")

Division 23134: Average Power = 132.25 (players=345)
Division 23190: Average Power = 132.04 (players=354)
Division 23164: Average Power = 131.89 (players=342)
Division 22921: Average Power = 130.25 (players=337)
Division 22994: Average Power = 130.19 (players=323)
Division 23081: Average Power = 129.95 (players=317)
Division 23117: Average Power = 128.54 (players=340)
Division 22924: Average Power = 128.42 (players=352)
Division 22964: Average Power = 128.35 (players=363)
Division 23094: Average Power = 127.97 (players=346)
Division 23193: Average Power = 127.64 (players=311)
Division 23145: Average Power = 127.27 (players=348)
Division 22972: Average Power = 126.83 (players=308)
Division 22982: Average Power = 126.48 (players=351)
Division 22951: Average Power = 126.27 (players=361)
Division 23176: Average Power = 125.81 (players=341)
Division 22930: Average Power = 125.55 (players=345)
Division 23060: Average Power = 125.41 (players=348)
Division 22966: Average Power = 124.94 (player

In [2]:
import json
import os


def calculate_average_power_all_divisions(file_paths):
    # Словник для зберігання сумарної сили та кількості гравців у всіх дивізіонах
    total_power = 0
    total_players = 0

    for file_path in file_paths:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Проходимо по всіх командах у файлі
        for team_id, team_data in data.get("teams", {}).items():
            players = team_data.get("players", {})

            # Проходимо по всіх гравцях команди
            for player_id, player_data in players.items():
                power = player_data.get("power")
                if power is not None:
                    total_power += power
                    total_players += 1

    # Обчислення середньої сили для всіх дивізіонів
    if total_players > 0:
        average_power = total_power / total_players
    else:
        average_power = 0

    return average_power


# Список файлів з країнами
country_files_dir = "/home/yevhen/Проекти/butsa/json/cleaned_responses/"
country_files = [
    os.path.join(country_files_dir, file)
    for file in os.listdir(country_files_dir)
    if file.endswith(".txt")
]

# Обчислення середньої сили для всіх дивізіонів
average_power_all_divisions = calculate_average_power_all_divisions(country_files)

# Виведення результату
print(f"Середня сила всіх дивізіонів: {average_power_all_divisions:.2f}")

Середня сила всіх дивізіонів: 108.18
